In [ ]:
import pandas as pd
import geopandas as gpd
import os, shutil

In [ ]:
%%capture
#### Steps for using AOI

## selected area of interest
## Reproject to quad file projection
aoi = gpd.read_file(r'D:\polygons_or_points.gpkg')
state = 'WA'
year = '2016'

## USGS quad files (DOQQ) from 
## https://www.fsa.usda.gov/programs-and-services/aerial-photography/imagery-programs/naip-qq-and-photocenter-shapefiles
quad_path = 'https://www.fsa.usda.gov/Assets/USDA-FSA-Public/usdafiles/APFO/imagery-programs/zips/naip'
## EPSG:4269
quad = gpd.GeoDataFrame.from_file(os.path.join(quad-path, year,state, 'naip' + year[2:] + 'qq.zip')

## AOI files need to be the same projection as the Quad file, else this will not work correctly
quad_intersection = gpd.sjoin(quad,aoi, how="right", op='intersects')

## The version of NAIP, verify using the USGS quad file, string in this format
state_year = state + '_' + year


q = quad_intersection.iloc[0:,1:26]
q = pd.DataFrame(q).drop_duplicates()
q = pd.DataFrame(q).drop_duplicates()
q.reset_index(inplace=True)
q.to_string()

In [ ]:
s3_NAIP = r'https://prd-tnm.s3.amazonaws.com/StagedProducts/NAIP/'+state_year+'/'
q['jpfilename'] = q.FileName.str[:-3]+'jp2'
q['ProductLink'] = s3_NAIP+q.APFONAME.str[:-2]+'/'+q.jpfilename

In [ ]:
for row in q.ProductLink:
    print(row)
    
len(q)

In [ ]:
# Optional to save list to csv
#q.to_csv('additional.csv')

In [ ]:
# Download the data directly
import requests

NAIP_PATH = os.path.join(r'D:\naip2016')

# For each row
for i, row in q.iterrows():
    if os.path.exists(os.path.join(NAIP_PATH, row.jpfilename)): 
        print'Skipping ' + row.jpfilename + ' as it already exists'
    else:
        response = requests.get(row.ProductLink)
        # If the response status code is fine (200)
        if response.status_code == 200:
            filename = os.path.join(NAIP_PATH,row.jpfilename)
            
            # Download the file 
            response = requests.get(row.ProductLink, stream=True)

            with open(filename, 'wb') as output:
                shutil.copyfileobj(response.raw, output)
                print filename + ' downloaded'